In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [35]:
# Importing the dataset
df = pd.read_csv("../../archive/Churn_Modelling.csv")

In [29]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Looking at customers at a bank (They have measuring unusual churn rates: people are leaving the bank)
# 1: person stayed in the bank
# 0: person left the bank
# We have to predict models of potential people who might leave the bank!
# We will use indepedent variables to predict whether people will leave the bank

In [39]:
# Making Y-Variable
y = df['Exited']

# Including all but one column
Xs = df.iloc[:, 3:13]

In [41]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_geo = LabelEncoder()
Xs.loc[:, "Geography"] = labelencoder_X_geo.fit_transform(Xs.loc[:, "Geography"])
labelencoder_X_gender = LabelEncoder()
Xs.loc[:, "Gender"] = labelencoder_X_gender.fit_transform(Xs.loc[:, "Gender"])

onehotencoder = OneHotEncoder(categorical_features=[Xs.columns.get_loc("Geography")])
Xs = onehotencoder.fit_transform(Xs).toarray()

In [45]:
# Removing one of the dummy variables for the country
Xs = Xs[:, 1:]

In [47]:
# Splitting the data set into training and testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=0)

In [48]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)

# We are only using the transform and not the fit transform because we already fit the model
X_test = sc_X.transform(X_test)

In [49]:
# Visual dataframe of the data
pd.DataFrame(data=Xs[0:,0:]).head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88
1,0.0,1.0,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58
2,0.0,0.0,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57
3,0.0,0.0,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63
4,0.0,1.0,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10


In [50]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [51]:
# Initializing the ANN
classifier = Sequential()

In [ ]:
# We will be using the rectifier function for the hidden layers
# We will be using the sigmoid function for the output layer

In [ ]:
# One of the tricks (or art) is to decide the number of input layers we would need
# A rule of thumb is make the # of nodes the avg. of the input and output layers
# In our case, we have 11 input layers and 1 output layer (1+11)/2 = 6 nodes


In [52]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))

In [53]:
# Adding the second hidden layer
# Do not need the input_dim bc we will have created the first layer, thus, it knows what to expect
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

In [54]:
# Adding the output layer
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

In [55]:
# If you are output layer would output 3, here's an example of it 
# classifier.add(Dense(units=3, kernel_initializer='uniform', activation='sufmaxed'))

In [56]:
# Compliling the ANN 
# The lost function for a logistic regression is the "logarithmic loss"
# For binary outcomes -> binary_crossentropy
# For multi-outcomes -> categorical_crossentropy
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [57]:
# Fitting our model
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 2s 193us/step - loss: 0.4929 - acc: 0.7959
Epoch 2/100
8000/8000 [==============================] - 1s 177us/step - loss: 0.4306 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 274us/step - loss: 0.4239 - acc: 0.7961
Epoch 4/100
8000/8000 [==============================] - 2s 191us/step - loss: 0.4174 - acc: 0.8265
Epoch 5/100
8000/8000 [==============================] - 2s 191us/step - loss: 0.4109 - acc: 0.8337
Epoch 6/100
8000/8000 [==============================] - 2s 189us/step - loss: 0.4074 - acc: 0.8340
Epoch 7/100
8000/8000 [==============================] - 1s 166us/step - loss: 0.4052 - acc: 0.8350
Epoch 8/100
8000/8000 [==============================] - 1s 178us/step - loss: 0.4039 - acc: 0.8352
Epoch 9/100
8000/8000 [==============================] - 1s 175us/step - loss: 0.4019 - acc: 0.8337
Epoch 10/100
8000/8000 [==============================] - 1s 169us/step - loss: 0.4012 - acc: 0.8340

In [58]:
# Predicting the training set
y_pred = classifier.predict(X_test)

In [61]:
# Have to create binary outcomes from the prob. of the logisic regression
y_pred = (y_pred > 0.5)

In [62]:
y_pred[0] # This customer has a 20% chance that he/she will leave the bank

array([False])

In [ ]:
# Use the model on the customer of the bank (rank the prob. of the highest to lowest)
# Segment the people who are leaving, use data mining techniques to find why customer will leave the bank!


In [63]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [65]:
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

total = tn+fp+fn+tp
accuracy = (tn+tp)/total
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = 2*precision*recall/(precision+recall)


print(
    "true pos: {0}\n"
    "false pos: {1}\n"
    "true neg: {2}\n"
    "false neg: {3}\n".format(tp, fp, tn, fn))

print("""
Out of {0} reviews, the model got {1} correct,
Accuacy is: {2:.2f}%
Precision is: {3:.2f}%
Recall is: {4:.2f}%
F1 Score is: {5:.2f}%""".format(total, tn+tp, accuracy, precision, recall, f1_score))

true pos: 173
false pos: 57
true neg: 1538
false neg: 232


Out of 2000 reviews, the model got 1711 correct,
Accuacy is: 0.86%
Precision is: 0.75%
Recall is: 0.43%
F1 Score is: 0.54%
